In [1]:
%load_ext autoreload
%autoreload 2

# Readme
Change dataset, crop_id, obj_id only

# 1. Crop images

In [3]:
import glob
from src.model.cnos_foundpose import cnos_foundpose, cnos_different_thresholds


dataset = "icbin"
obj_id = 1
rgb_paths = sorted(glob.glob(f"datasets/bop23_challenge/datasets/{dataset}/test/{obj_id:06d}/rgb/*.png"))

In [3]:
# i = 2
# scene_id = int(rgb_paths[i].split("/")[-3])
# frame_id = int(rgb_paths[i].split("/")[-1].split(".")[0])
# cnos_foundpose(rgb_path=rgb_paths[i], scene_id=scene_id, frame_id=frame_id, obj_id=obj_id, dataset=dataset)

In [6]:
# for i in range(len(rgb_paths)):
from tqdm import trange


for i in trange(30,len(rgb_paths)):
    scene_id = int(rgb_paths[i].split("/")[-3])
    frame_id = int(rgb_paths[i].split("/")[-1].split(".")[0])
    cnos_different_thresholds(rgb_path=rgb_paths[i], scene_id=scene_id, frame_id=frame_id, obj_id=obj_id, dataset=dataset)

  0%|          | 0/26 [00:00<?, ?it/s][2024-09-01 19:36:36,226] [INFO] root: Loading SAM model from datasets/bop23_challenge/pretrained/segment-anything
[2024-09-01 19:36:39,125] [INFO] root: Init CustomSamAutomaticMaskGenerator done!
Using cache found in /home/cuong.van-dam/.cache/torch/hub/facebookresearch_dinov2_main
[2024-09-01 19:36:41,688] [INFO] dinov2: using MLP layer as FFN
100%|██████████| 104/104 [00:01<00:00, 52.15it/s]

File cnos_foundpose_analysis/output_npz/000001_000030_cnos_05.json does not exist.
File cnos_foundpose_analysis/output_npz/000001_000030_cnos_05.npz does not exist.



[2024-09-01 19:36:49,066] [INFO] root: Loading detections...
100%|██████████| 15/15 [00:03<00:00,  4.32it/s]

File cnos_foundpose_analysis/output_npz/000001_000030_cnos_04.json does not exist.
File cnos_foundpose_analysis/output_npz/000001_000030_cnos_04.npz does not exist.



[2024-09-01 19:36:54,228] [INFO] root: Loading detections...
100%|██████████| 30/30 [00:12<00:00,  2.49it/s]

...


[2024-09-01 21:19:38,910] [INFO] root: Loading detections...
100%|██████████| 32/32 [00:13<00:00,  2.39it/s]

File cnos_foundpose_analysis/output_npz/000001_000055_cnos_03.json does not exist.
File cnos_foundpose_analysis/output_npz/000001_000055_cnos_03.npz does not exist.



[2024-09-01 21:19:57,436] [INFO] root: Loading detections...
100%|██████████| 84/84 [01:25<00:00,  1.01s/it]


File cnos_foundpose_analysis/output_npz/000001_000055_cnos_02.json does not exist.
File cnos_foundpose_analysis/output_npz/000001_000055_cnos_02.npz does not exist.


[2024-09-01 21:21:31,607] [INFO] root: Loading detections...
100%|██████████| 26/26 [1:47:53<00:00, 249.00s/it]


In [5]:
import math
x = float(2)
if not math.isnan(x):
    print("Flase")

Flase
